This document compares parameter sheets for the IPPR tax benefit model and returns a list of the differences between them.

Currently it only compares for a particular year, and ignores the following sheets:
- Summary
- ReadMe
- Monthly (TODO)
- Regional (TODO)
- National (TODO)
- TaxYear (TODO)
- Data 
- Uprating 
- TakeUp 
- CalcEmpChangeQuints 

Future to dos:
- monthly - taxyear comparisons
- multiple sheets in one go

In [1]:
import pandas as pd
import numpy as np

In [14]:
#   LOCATIONS OF FILES TO COMPARE
##  update/edit as appropriate

BASE_FILE = 'C:\\TaxBenefit\\v02_31\\parameters\\0v02_31_Base.xlsx'
CHECK_FILE = 'C:\\TaxBenefit\\v02_31\\parameters\\ae_wna_fulluc.xlsx'
OUTPUT_FILE = 'C:\\TaxBenefit\\v02_31\\parameters\\COMPARE_FILE.xlsx' # change this to whatever you want the name to be
YEAR = "2026-27"

In [7]:
# loading in base file
xls_base = pd.ExcelFile(BASE_FILE)

# loading in sheet names
all_sheets = xls_base.sheet_names
macro_parameters = all_sheets[2:8] # monthly - taxyear
data_parameters = all_sheets[8:10]
takeup = all_sheets[11]
employment_change_calculations = all_sheets[12]
policy_parameters = all_sheets[13:35] # employment_change - scottishchildpayment
policy_parameters.insert(0, all_sheets[10])
list_parameters = all_sheets[35:] # lists

xls_compare_0 = pd.ExcelFile(CHECK_FILE)

In [18]:
# CYCLING THROUGH POLICY SHEETS FOR 1 COMPARE VALUE

def policy_base_comparison(base_file, compare_file):
    """Takes a pandas ExcelFile version of a base file and comparison file
    and outputs a dataframe showing the differences
    
    Currently DOES NOT CHECK Monthly - UPrating, TakeUp or CalcChangeEmployment
    """

    data_diff = {'Name' : [], 'BaseValue' : [], 'CompareValue' : []}
    
    for sheet in policy_parameters:
        # dealing with policy parameters
        df_base = base_file.parse(sheet).fillna("missing")
        base_name_year = df_base[["Name", YEAR]]
        bny = base_name_year.drop([0,1])

        df_compare = compare_file.parse(sheet).fillna("missing")
        compare_name_year = df_compare[["Name", YEAR]]
        cny = compare_name_year.drop([0,1])

        # removing additional rows - may break if basefile has been messed with
        if len(cny.index) > len(bny.index):
            drop_rows = len(cny.index) - len(bny.index)
            cny = cny[0: -drop_rows]

        # creating a matrix of bools where False indicates difference
        comparing_sheet = bny.values == cny.values

        # identifying where different
        rows, cols = np.where(comparing_sheet==False)

        # logging difference in dataframe
        for item in zip(rows, cols):
            n = bny.iloc[item[0]][0]
            vbase = bny.iloc[item[0]][1]
            vcompare = cny.iloc[item[0]][1]
            data_diff['Name'].append(n)
            data_diff['BaseValue'].append(vbase)
            data_diff['CompareValue'].append(vcompare)
        
    for sheet in list_parameters:
        df_base = base_file.parse(sheet).fillna("missing")
        df_base.drop(['Name1', 'Name2', 'Name3'], axis=1, inplace=True)

        df_compare = compare_file.parse(sheet).fillna("missing")
        df_compare.drop(['Name1', 'Name2', 'Name3'], axis=1, inplace=True)
        
        # creating a matrix of bools where False indicates difference
        comparing_sheet = df_base == df_compare

        # identifying where different
        rows, cols = np.where(comparing_sheet==False)

        # logging difference in dataframe
        for item in zip(rows, cols):
            n = df_base.iloc[item[0]][0] + "..." + df_base.columns[item[1]]
            vbase = df_base.iat[item[0], item[1]]
            vcompare = df_compare.iat[item[0], item[1]]
            data_diff['Name'].append(n)
            data_diff['BaseValue'].append(vbase)
            data_diff['CompareValue'].append(vcompare)        

    df_differences = pd.DataFrame.from_dict(data_diff)
    
    return df_differences.set_index('Name')

Checking file: C:\TaxBenefit\v02_31\parameters\ae_wna_fulluc.xlsx, year: 2026-27
Results saved at C:\TaxBenefit\v02_31\parameters\COMPARE_FILE.xlsx
                                                   BaseValue CompareValue
Name                                                                     
General.TakeUp.UniversalCredit                          0.86            1
IncomeTax.PersonalAllowance.Main                       13910          500
IncomeTax.Earnings.Rate.Over                              45           48
IncomeTax.SavingsAllowance.Basic                        1000            0
IncomeTax.SavingsAllowance.Reduced                       500            0
IncomeTax.DividendAllowance.Main                        2000            0
IncomeTax.Dividend.Rate.1                                  8           20
IncomeTax.Dividend.Rate.2                                 33           40
IncomeTax.Dividend.Rate.Over                              38           45
IncomeTax.Savings.BandLimit.1         

In [ ]:
# Results here

df1 = policy_base_comparison(xls_base, xls_compare_0)
print(f"Checking file: {CHECK_FILE}, year: {YEAR}")
df1.to_excel(OUTPUT_FILE)
print(f"Results saved at {OUTPUT_FILE}")
print(df1)